In [1]:
pwd

'c:\\Users\\atsuy\\Documents'

In [2]:
#yolov5ディレクトリにいろいろ関数がおかれているのでこのディレクトリで操作する
%cd ../yolov5

c:\Users\atsuy\yolov5


In [3]:
#yolov5　cocoモデルで80種類の物体を検出　”ｑ”押下で終了
!python detect.py --source 0

detect: weights=yolov5s.pt, source=0, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-36-gc09fb2a torch 1.11.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)

0: 480x640 1 person, Done. (0.225s)
0: 480x640 1 person, Done. (0.209s)
0: 480x640 1 person, Done. (0.209s)
0: 480x640 1 person, Done. (0.205s)
0: 480x640 1 person, Done. (0.198s)
0: 480x640 1 person, Done. (0.206s)
0: 480x640 1 person, Done. (0.192s)
0: 480x640 1 person, Done. (0.183s)
0: 480x640 1 person, Done. (0.183s)
0: 480x640 1 person, Done. (0.193s)
0: 480x640 1 person, Done. (0.206s)
0: 480x640 

In [4]:
#tourch hubを使って学習済みモデルをmodelに格納
import torch
#cocomodelを読み込む場合
model = torch.hub.load("../yolov5","yolov5s",source='local')
#ocd検出モデルを読み込む場合
#model = torch.hub.load('','custom',path='ocd_20220413.pt',source='local',force_reload=True)

YOLOv5  v6.1-36-gc09fb2a torch 1.11.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
#静止画で検出
import cv2
import matplotlib.pyplot as plt
imgs=cv2.imread('./data/images/ocd_1.jpg')
results = model(imgs)
print(results.xyxy[0])
results.show()
"""
plt.imshow(results)
plt.show()"""

tensor([], size=(0, 6))


'\nplt.imshow(results)\nplt.show()'

In [6]:
#webカメラで検出
import cv2
camera = cv2.VideoCapture(0) #外部入力カメラを使用する場合0を1に変更
#camera = cv2.VideoCapture('./data/images/ocd.mp4')
while True:
    ret, imgs = camera.read()
    if not ret:
        break
    h,w,ch=imgs.shape
#画面を10*10分割して1/10のところから9/10のところまで切り抜き
    im_cropped=imgs[round(h/10):round(h/10)*9, round(w/10):round(w/10)*9,:]
    results = model(im_cropped)
    for *box, conf, cls in results.xyxy[0]:
        s = model.names[int(cls)]+":"+'{:.1f}'.format(float(conf)*100)
        cv2.rectangle(im_cropped,
                      (int(box[0]),int(box[1])),
                      (int(box[2]),int(box[3])),color=(0,255,0),thickness=4)
        cv2.putText(im_cropped, s, (int(box[0]),int(box[1])-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1.0,
                   color=(0,0,255),thickness =2)
    cv2.imshow('tfcc',imgs)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
camera.release()    
cv2.destroyAllWindows()

KeyboardInterrupt: 

#GUIのレイアウト設定

In [2]:
import PySimpleGUI as sg
sg.theme('SystemDefault')
values = [0, 1, 2]#cameraの番号設定
layout = [
   [sg.Image(key='img1',), sg.Image(key='img2',)],
   [[sg.Text('select camera'),sg.Listbox(values, size=(10, 5))]],
    [[sg.Button('Start', size=(10, 1)), sg.Button('Stop', size=(10,1))]],
    [sg.Button('Exit', size=(10, 1))],]
#記録イベントのためにstartedという関数を定義               
started= False

In [3]:
#tourch hubを使って学習済みモデルをmodelに格納
import torch
#cocomodelを読み込む場合
model = torch.hub.load("../yolov5","yolov5s",source='local')
#ocd検出モデルを読み込む場合
#model = torch.hub.load('','custom',path='ocd_20220413.pt',source='local',force_reload=True)

#torchhubモデルの検出時のパラメーター設定
"""model.conf=0.4
model.iou=0.45
model.multi_label=False
model.max_det=5"""

import PySimpleGUI as sg
#すべての警告の非表示
import warnings
warnings.simplefilter('ignore')

#value = [0, 1, 2]#cameraの番号設定
threshould=0.4 #閾値の初期設定
#カメラ番号入力のポップアップ 外部入力なら1か2
cameranum = sg.popup_get_text("input camera number(0,1,2)", title="camera number", default_text="1")
cameranum =int(cameranum)
#GUIの初期設定
sg.theme('DarkBlue15')
layout = [
   [sg.Image(key='img1',), sg.Image(key='img2',)],
   #[[sg.Text('select camera'),sg.Listbox(value, size=(10, 3),key=('cameranum'))]],
   [[sg.Text('confidence thredshold'),sg.Slider(range=(0.1,1.0),default_value=0.4, resolution=0.1 ,orientation='h',
   size=(20,2),enable_events=True, key='slider',)]],
    [[sg.Button('Start', size=(10, 1)), sg.Button('Stop', size=(10,1))]],

    [sg.Button('Exit', size=(10, 1))],]
#記録イベントのためにstartedという関数を定義               
started= False
#webカメラで検出
import cv2
import time
import datetime as dt


#camera = cv2.VideoCapture('./data/images/ocd.mp4')
#pysimpleGUIのwindow定義
window = sg.Window("webカメラ画面", location=(100,100),layout=layout, size=(1000,1000),resizable=True,finalize=True)
event, values = window.read(timeout=20)
#cameranum = int(values['cameranum'])
#if event == ['caneranum']:
#    cameranum=int(values['cameranum'])
cap = cv2.VideoCapture(cameranum) #外部入力カメラを使用する場合0を1に変更
# ビデオ記録用の変数定義
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('result_{}.mp4'.format(dt.datetime.now()), fourcc, fps, (w, h))
while True:
    ret, imgs = cap.read()
    if not ret:
        break
    h,w,ch=imgs.shape
# キャプチャーした画像をpngに変換
    image1 = cv2.imencode('.png', imgs)[1].tobytes()                   
# Imageの内容を更新
    window['img1'].update(data=image1)#web cameraの画像を直接表示
    event, values = window.read(timeout=0)
    """if event == ['caneranum']:
        cameranum=int(values['cameranum'])
        cap=cv2.VideoCapture(cameranum)
        ret,img = cap.read()"""

    if values['slider']:
      threshould=values['slider']
      model.conf=threshould

    if event == 'Start':
        started = True
    if started == True:
        #画面を10*10分割して1/10のところから9/10のところまで切り抜き
        im_cropped=imgs[round(h/10):round(h/10)*9, round(w/10):round(w/10)*9,:]
        results = model(im_cropped)
        for *box, conf, cls in results.xyxy[0]:
            s = model.names[int(cls)]+":"+'{:.1f}'.format(float(conf)*100)
            cv2.rectangle(im_cropped,
                      (int(box[0]),int(box[1])),
                      (int(box[2]),int(box[3])),color=(0,255,0),thickness=4)
            cv2.putText(im_cropped, s, (int(box[0]),int(box[1])-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1.0,
                   color=(0,0,255),thickness =2)          
    # キャプチャーした画像をpngに変換
            image2 = cv2.imencode('.png', im_cropped)[1].tobytes()
    # Imageの内容を更新
            window['img2'].update(data=image2)       

    if event =='Stop':
        started = False
        out.release()

    if event == 'Exit' or event == sg.WIN_CLOSED:
        break

cap.release()    
window.close()

YOLOv5  v6.1-36-gc09fb2a torch 1.11.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
#tourch hubを使って学習済みモデルをmodelに格納
import torch
#cocomodelを読み込む場合
#model = torch.hub.load("../yolov5","yolov5s",source='local')
#ocd検出モデルを読み込む場合
model = torch.hub.load('','custom',path='ocd_20220413.pt',source='local',force_reload=True)

#torchhubモデルの検出時のパラメーター設定
"""model.conf=0.4
model.iou=0.45
model.multi_label=False
model.max_det=5"""

import PySimpleGUI as sg
#すべての警告の非表示
import warnings
warnings.simplefilter('ignore')

#value = [0, 1, 2]#cameraの番号設定
threshould=0.4 #閾値の初期設定
#カメラ番号入力のポップアップ 外部入力なら1か2
cameranum = sg.popup_get_text("input camera number(0,1,2)", title="camera number", default_text="1")
cameranum =int(cameranum)
#GUIの初期設定
sg.theme('DarkBlue15')
layout = [
   [sg.Image(key='img1',), sg.Image(key='img2',)],
   #[[sg.Text('select camera'),sg.Listbox(value, size=(10, 3),key=('cameranum'))]],
   [[sg.Text('confidence thredshold'),sg.Slider(range=(0.1,1.0),default_value=0.4, resolution=0.1 ,orientation='h',
   size=(20,2),enable_events=True, key='slider',)]],
    [[sg.Button('Start', size=(10, 1)), sg.Button('Stop', size=(10,1))]],

    [sg.Button('Exit', size=(10, 1))],]
#記録イベントのためにstartedという関数を定義               
started= False
#webカメラで検出
import cv2
import time
import datetime as dt


#camera = cv2.VideoCapture('./data/images/ocd.mp4')
#pysimpleGUIのwindow定義
window = sg.Window("webカメラ画面", location=(100,100),layout=layout, size=(1000,1000),resizable=True,finalize=True)
event, values = window.read(timeout=20)
#webカメラで検出
import cv2
import time
import datetime as dt
cap = cv2.VideoCapture(0) #外部入力カメラを使用する場合0を1に変更
# ビデオ記録用の変数定義
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('result_{}.mp4'.format(dt.datetime.now()), fourcc, fps, (w, h))
#camera = cv2.VideoCapture('./data/images/ocd.mp4')
#pysimpleGUIのwindow定義
window = sg.Window("webカメラ画面", location=(100,100),layout=layout, size=(1000,1000),resizable=True,finalize=True)
while True:
    ret, imgs = cap.read()
    if not ret:
        break
    h,w,ch=imgs.shape
#画面を10*10分割して1/10のところから9/10のところまで切り抜き
    im_cropped=imgs[round(h/10):round(h/10)*9, round(w/10):round(w/10)*9,:]
    results = model(im_cropped)
    for *box, conf, cls in results.xyxy[0]:
        s = model.names[int(cls)]+":"+'{:.1f}'.format(float(conf)*100)
        cv2.rectangle(im_cropped,
                      (int(box[0]),int(box[1])),
                      (int(box[2]),int(box[3])),color=(0,255,0),thickness=4)
        cv2.putText(im_cropped, s, (int(box[0]),int(box[1])-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1.0,
                   color=(0,0,255),thickness =2)
# キャプチャーした画像をpngに変換
    image1 = cv2.imencode('.png', imgs)[1].tobytes()                   
# Imageの内容を更新
    window['img1'].update(data=image1)#web cameraの画像を直接表示
    event, values = window.read(timeout=0)
    if event == 'Start YOLOV5':
        started = True
            
    if started == True:
    #画像を録画用に保存
            out.write(im_cropped)                   
    # キャプチャーした画像をpngに変換
            image2 = cv2.imencode('.png', im_cropped)[1].tobytes()
    # Imageの内容を更新
            window['img2'].update(data=image2)       

    if event =='Stop Log':
            started = False
            out.release()

    if event == 'Exit' or event == sg.WIN_CLOSED:
            break

camera.release()    
cv2.destroyAllWindows()

YOLOv5  v6.1-36-gc09fb2a torch 1.11.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7023610 parameters, 0 gradients
Adding AutoShape... 


SystemExit: 

In [ ]:
if results == True:
     print('yes')
else:
    print('no results')

no results
